In [1]:
import requests
from bs4 import BeautifulSoup, NavigableString, Tag
import re
import pandas as pd
import numpy as np
import json

In [2]:
from lxml.html import fromstring
import requests
from itertools import cycle
import traceback
import yfinance as yf

def get_proxies(n_proxies):
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = []
    for i in parser.xpath('//tbody/tr')[:n_proxies]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.append(proxy)
    return proxies


def proxy_get(url, n_proxies=100):
    proxies = get_proxies(n_proxies)
    proxy_pool = cycle(set(proxies))
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    for i in range(1, n_proxies+1):
        # Get a proxy from the pool
        proxy = next(proxy_pool)
        # print("Request #%d" % i)
        try:
            response = requests.get(url, proxies={"http": proxy, "https": proxy}, headers=headers)
            return response
        except Exception as e:
            pass
    return requests.get(url)

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
## First we need components of the Stoxx600
## however there seems to be no available list of tickers 
## instead we need to find the name => ticker =>download historical data

## This is the first approach, which yields 136 tickers, 
## please skip to cell 9 for the final approach of 412 tickers

## investing.com has names of the company, and each has a href which leads to ticker/ symbol
html_index = r"https://uk.investing.com/indices/stoxx-600-components"
r = proxy_get(html_index)
index_soup = BeautifulSoup(r.text, 'html.parser')

names_list = set()
href_list = set()
for trs in index_soup.find_all('tbody'):
    for a in trs.find_all('a'):
        if 'equities' in a['href']:
            names_list.add(*a.contents)
            href_list.add(a['href'])

In [5]:
len(names_list)


612

In [6]:
with open("stoxx600_component_names.txt", "w") as text_file:
    text_file.write(",".join(names_list))

In [7]:
## join with a file that supposed to have all yahoo finance stock tickers
## seems the list is a bit short 
names2ticker = pd.read_csv("name2ticker.csv", encoding='ISO-8859-1')
europe_countries = ["Albania","Andorra","Armenia","Austria","Azerbaijan","Belarus","Belgium","Bosnia and Herzegovina","Bulgaria","Croatia","Cyprus","Czech Republic","Denmark","Estonia","Finland","France","Georgia","Germany","Greece","Hungary","Iceland","Ireland","Italy","Kazakhstan","Latvia","Liechtenstein","Lithuania","Luxembourg","Macedonia","Malta","Moldova","Monaco","Montenegro","Netherlands","Norway","Poland","Portugal","Romania","Russia","San Marino","Serbia","Slovakia","Slovenia","Spain","Sweden","Switzerland","Turkey","Ukraine","United Kingdom","Vatican City"]
names2ticker.loc[(names2ticker['Name'].isin(names_list))&(names2ticker['Country'].isin(europe_countries))]

,Ticker,Name,Exchange,Category Name,Country
974,WDI.DE,Wirecard AG,GER,Business Services,Germany
995,MT.AS,ArcelorMittal,AMS,Steel & Iron,Netherlands
1329,SAND.ST,Sandvik AB,STO,Diversified Machinery,Sweden
1860,ARRB.DE,ArcelorMittal,GER,NaN,Germany
3205,BN.PA,Danone,PAR,Food - Major Diversified,France
...,...,...,...,...,...
69262,SANF.MI,Sanofi,MIL,Drug Manufacturers - Major,Italy
69471,ML.SW,Michelin,EBS,NaN,Switzerland
69610,RUI.PA,Rubis,PAR,Gas Utilities,France
69666,FRA.SW,Fraport AG,EBS,NaN,Switzerland


In [8]:
## we'll save this for now, let see if we can get ticker by going to each href link
fornow_li_stoxx600 = names2ticker.loc[(names2ticker['Name'].isin(names_list))&(names2ticker['Country'].isin(europe_countries))]['Ticker'].tolist()

## Change from MD to Code to run,

## rather than joining to the file, we can go to each href and obtain ticker,
## would need proxies and investing.com bans traffic even with headers/ low request frequency
## this found to be very slow, due to free proxies 

new_new_li_yhoo_tickers = []
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
for href in href_links[::-1]:
    url_ticker = f"https://www.investing.com{href}"
    r = proxy_get(url_ticker)
    ticker_index_soup = BeautifulSoup(r.text, 'html.parser')
    for sth in ticker_index_soup.find_all("div", {"class":"instrumentHead"}):
        for sth_else in sth.find_all("h1"):
            t = sth_else.contents[0]
            ticker = re.search(r'\((.*?)\)',t).group(1)
            # print(re.search(r"\[([A-Za-z0-9_]+)\]", t))
        for exchange in sth.find_all("div", {"class":"exchangeDropdownContainer"}):
            location = exchange.find("i").contents[0]
    try:  
        ex_suffix = city2ticker.loc[(city2ticker['Market'].str.contains(location))|(city2ticker['City'].str.contains(location))|(city2ticker['Country'].str.contains(location))].head(1)['Suffix'].values[0]
        print(f"{ticker}{ex_suffix}")
        
        
        new_new_li_yhoo_tickers.append(f"{ticker}{ex_suffix}")
    
    
    except:
        
        print(f"fail {ticker} {location}")
        
        
        
        
        

In [9]:
## The successful method
## finding out hidden deep in stoxx website, a list of component companies name
## New approach would be search for autocomplete from yahoo finance then download with ticker
company_names = pd.read_csv('SXXGR.csv')
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
tickers = []
for name in company_names['Company'].tolist():
    yhoo_url = f"http://d.yimg.com/autoc.finance.yahoo.com/autoc?query={name}&region=1&lang=en&callback=YAHOO.Finance.SymbolSuggest.ssCallback"
    r = requests.get(yhoo_url, headers=headers)
    try:
        ticker = [x for x in json.loads(re.search(r'\((.*?)\)',r.text).group(1))['ResultSet']['Result'] if x['exch']!='PNK' and x['exchDisp']!='OTC Markets' and x['exchDisp']!='NYSE' ]
        print(f"{name}: {ticker[0]['symbol']}")
        tickers.append(ticker[0]['symbol'])
    except Exception as e:
        print(f"fail {name}")





NESTLE: NESM.KL
fail ROCHE HLDG P
NOVARTIS: 0QLR.IL
ASML HLDG: ASML
SAP: SPNS
ASTRAZENECA: AZN.L
SANOFI: SNY
fail NOVO NORDISK B
fail LINDE
LVMH MOET HENNESSY: 0HAU.IL
TOTAL: TOT.TI
GLAXOSMITHKLINE: GSK.L
HSBC: HSOAX
SIEMENS: SIE.DE
BRITISH AMERICAN TOBACCO: BTIIX
ALLIANZ: ALV.TI
DIAGEO: DGE.L
BP: BPY
UNILEVER NV: UAL
BAYER: BMW.DE
fail L'OREAL
ROYAL DUTCH SHELL A: R6C.HM
AIR LIQUIDE: 0NWF.IL
IBERDROLA: IBE1.SG
UNILEVER PLC: ULTA
ENEL: ENEL.MI
fail RECKITT BENCKISER GRP
RIO TINTO: RIOT
SCHNEIDER ELECTRIC: SCHN.TI
DEUTSCHE TELEKOM: 0MPH.IL
VINCI: DG.PA
BASF: BASF.TI
ZURICH INSURANCE GROUP: ZFIN.SG
ADIDAS: ADSG-U.TI
DANONE: BSN.HM
fail RELX PLC
BNP PARIBAS: ^BNPIC52T
AIRBUS: AIR.BE
ANHEUSER-BUSCH INBEV: 1NBA.HA
AXA: AXA.TI
Kering: KER.PA
VODAFONE GRP: VOD
NATIONAL GRID: NGG22.NYM
fail BHP GROUP PLC.
PHILIPS: PHIA.TI
fail BCO SANTANDER
Prosus: PRX.AS
ESSILORLUXOTTICA: ESL.SG
fail UBS GROUP
PRUDENTIAL: PBIP
ABB: LALDX
LONZA: LONN.SW
SAFRAN: SAF.PA
MUENCHENER RUECK: MUV2.SG
ADYEN: ADYEN.AS


TECAN: TECN.SW
fail GALENICA SANTE AG
TRAVIS PERKINS: TPK.L
fail WDP
fail FABEGE
SIEMENS GAMESA: SGRE.VI
ELIA GROUP: E4S.BE
DECHRA PHARMACEUTICALS: DPH.L
fail INDUTRADE
GJENSIDIGE FORSIKRING: GJF.OL
TECHNIPFMC: FTI.PA
ITV: ITV.L
JD SPORTS FASHION: JD.L
fail ELEKTA B
fail EVOTEC
COFINIMMO: COFB.BR
fail TRELLEBORG B
SIG COMBIBLOC GROUP AG: SIGN.SW
GEORG FISCHER: 0QP4.IL
WEIR GRP: 42W.BE
Valmet: VALMT.HE
UNITE GROUP PLC: U1B.MU
TRYG: TRYG.CO
fail LUNDBERGFORETAGEN B
IAG: IAGAX
PROXIMUS: PROX.BR
WIRECARD: WDI.SG
AMPLIFON: AMP.MI
AVEVA GRP: 4A91.BE
IMI: IMIDX
GAMES WORKSHOP: GAW.L
fail HUSQVARNA B
TENARIS: TW11.SG
MERLIN PROPERTIES SOCIMI: MEQA.SG
THYSSENKRUPP: TKA.MU
BANKINTER: 0H7O.IL
ADP: ADP
HOLMEN B: 0XS9.IL
CENTRICA: CNA.L
HERA: HERAX
fail AAK
TRITAX BIG BOX REIT: BBOX.L
ACKERMANS & VAN HAAREN: ACKB.BR
fail HELVETIA HLDG
BAKKAFROST: BAKKA.OL
HISCOX: HSX.L
ILIAD: IL2.BE
fail TRAINLINE PLC
IPSEN: I7G.MU
MEGGITT: MGGT.L
SIGNIFY: LIGHT.AS
CEMBRA MONEY BANK: GEH.SG
fail SWECO 'B'
REMY COIN

In [10]:
## filter out possible US companies, not within stoxx 600 based on it must have an . extension
tickers = [ticker for ticker in tickers if '.' in ticker]
print(len(tickers))
tickers_str = " ".join(tickers)
new_data = yf.download(tickers=tickers_str,period='max')

412
[*********************100%***********************]  412 of 412 completed

33 Failed downloads:
- UN01.TI: 1d data not available for startTime=-2208988800 and endTime=1598279281. Only 100 years worth of day granularity data are allowed to be fetched per request.
- EOA.TI: 1d data not available for startTime=-2208988800 and endTime=1598279283. Only 100 years worth of day granularity data are allowed to be fetched per request.
- PUM.TI: 1d data not available for startTime=-2208988800 and endTime=1598279283. Only 100 years worth of day granularity data are allowed to be fetched per request.
- DCX.TI: 1d data not available for startTime=-2208988800 and endTime=1598279283. Only 100 years worth of day granularity data are allowed to be fetched per request.
- TELO.OL: 1d data not available for startTime=-2208988800 and endTime=1598279284. Only 100 years worth of day granularity data are allowed to be fetched per request.
- ORKO.OL: 1d data not available for startTime=-2208988800 and endTim

In [11]:
new_data['Close']

,0GRX.IL,0GWL.IL,0GZV.IL,0H13.IL,0H7O.IL,0HAU.IL,0HBT.IL,0IAH.IL,0IKJ.IL,0IXZ.IL,...,VWS.CO,WDI.SG,WIB.BE,WKL.AS,WO6.BE,WPP.L,WTB.L,YAR.OL,ZAL.DE,ZFIN.SG
Date,,,,,,,,,,,,,,,,,,,,,
1986-07-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-07-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-07-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-07-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1986-07-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-18,602.599976,274.500000,201.300003,230.199997,4.6230,384.825012,179.149994,127.025002,7.174,3.107,...,926.000000,1.2502,21.799999,69.879997,73.059998,607.599976,2335.0,384.0,67.839996,305.100006
2020-08-19,604.900024,274.750000,203.250000,230.800003,4.5925,386.875000,178.375000,126.125000,7.154,3.161,...,926.799988,1.3152,22.000000,70.580002,72.180000,613.400024,2372.0,385.0,68.660004,305.100006
2020-08-20,596.799988,273.500000,201.699997,232.199997,4.5825,383.549988,178.550003,124.199997,7.035,3.129,...,917.400024,1.2856,21.559999,70.339996,72.120003,604.599976,2332.0,378.5,68.220001,305.100006


In [12]:
import types
def imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            yield val.__name__
list(imports())

['builtins',
 'builtins',
 'requests',
 're',
 'pandas',
 'numpy',
 'json',
 'traceback',
 'yfinance',
 'types']

In [22]:
new_data.dropna(how='all', axis=1, inplace=True)
new_data['Close'].to_pickle('new_close_stoxx600.pkl')

In [13]:
## Stoxx 600, index price
stoxx = yf.Ticker('^STOXX')
stoxx = stoxx.history(period="max")

stoxx.to_pickle('stoxx600.pkl')

In [18]:
yf.__version__

'0.1.54'